In [1]:
# Import necessary libraries for data handling, visualization, and GIF creation
import os
import numpy as np
import xarray as xr
import imageio.v2 as imageio
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.colors import LinearSegmentedColormap, Normalize

# Define a custom color map to visualize wind speed with shades of blue
colors = ["#e0f7fa", "#b2ebf2", "#80deea", "#4dd0e1", "#26c6da",
          "#00bcd4", "#00acc1", "#0097a7", "#00838f", "#006064",
          "#004f5c", "#003a4a", "#002938", "#001921", "#000c12"]
custom_cmap = LinearSegmentedColormap.from_list("custom_blue_15", colors, N=15)

# Load datasets for wind direction and speed from specified file paths
dir_data = xr.open_dataset('../../data/th.nc')
speed_data = xr.open_dataset('../../data/vs.nc')

# Extract longitude and latitude arrays from the datasets
lon = dir_data['lon'].values
lat = dir_data['lat'].values

# Define the total number of arrows to be plotted in the quiver plot
num_arrows = 1000

# Define the list of specific days for which the visualization is to be generated
day_list = [32, 33, 34, 35, 36, 87, 88, 89, 90, 91]
num_frames = len(day_list)

# Calculate step size for downsampling based on the number of arrows
# This controls the arrow density by selecting evenly spaced points
step_lon = len(lon) // int(np.sqrt(num_arrows))
step_lat = len(lat) // int(np.sqrt(num_arrows))

# Create index arrays for sampling longitude and latitude values
lon_indices = np.arange(0, len(lon), step_lon)[:int(np.sqrt(num_arrows))]
lat_indices = np.arange(0, len(lat), step_lat)[:int(np.sqrt(num_arrows))]

# Generate a grid of sampled longitude and latitude points for plotting arrows
sample_lon, sample_lat = np.meshgrid(lon[lon_indices], lat[lat_indices])

# Set normalization for the color map to handle wind speed values in a given range
norm = Normalize(vmin=0, vmax=10)
print("Running...")

# Function to generate images for wind direction and speed for each selected day
def generate_images():
    os.makedirs('../images/color_quiver', exist_ok=True)  # Ensure output directory exists

    for day in day_list:
        # Extract wind speed and direction data for the specified day
        wind_speed = speed_data['wind_speed'].isel(day=day).values
        wind_dir = dir_data['wind_from_direction'].isel(day=day).values

        # Convert wind direction from degrees to radians for vector component calculation
        wind_rad = np.deg2rad(wind_dir)

        # Calculate U and V components of wind vectors (unit vectors for direction)
        U = np.sin(wind_rad)  # U component (east-west)
        V = np.cos(wind_rad)  # V component (north-south)

        # Sample U, V, and wind speed data at specified indices
        U_sample = U[lat_indices[:, None], lon_indices]
        V_sample = V[lat_indices[:, None], lon_indices]
        speed_sample = wind_speed[lat_indices[:, None], lon_indices]  # Wind speed for color mapping

        # Initialize the plot for the current day
        fig, ax = plt.subplots(figsize=(10, 7))

        # Create a Basemap projection for the region (USA and surroundings)
        m = Basemap(llcrnrlon=-123, llcrnrlat=20, urcrnrlon=-62, urcrnrlat=50,
                    projection='lcc', lat_1=33, lat_2=45, lat_0=39.5, lon_0=-98, ax=ax)

        # Draw map boundaries and continents with specified colors
        m.drawmapboundary(fill_color='#A6CAE0')  # Water color
        m.fillcontinents(color='#E1DCBD', lake_color='#A6CAE0', alpha=0.7)  # Land color
        m.drawcoastlines(color='#404040', linewidth=0.8)  # Coastline color and width
        m.drawcountries(color='#404040', linewidth=0.6)  # Country borders

        # Draw latitude and longitude lines with labels
        m.drawparallels(np.arange(20, 51, 10), labels=[1, 0, 0, 0], fontsize=8, color='#808080', linewidth=0.5)
        m.drawmeridians(np.arange(-120, -60, 10), labels=[0, 0, 0, 1], fontsize=8, color='#808080', linewidth=0.5)

        # Convert sampled longitude and latitude points to map projection coordinates
        x, y = m(sample_lon, sample_lat)

        # Create a quiver plot with arrows colored by wind speed
        q = m.quiver(x, y, U_sample, V_sample, speed_sample, cmap=custom_cmap, norm=norm, scale=40)

        # Add a color bar to indicate wind speed values
        plt.colorbar(q, label='Wind Speed (m/s)', orientation='vertical', fraction=0.046, aspect=10)

        # Set the title with the current day's information
        plt.title(f"Wind Direction Analysis - Day {day}", fontsize=14, pad=20, fontweight='bold')
        plt.tight_layout()  # Adjust layout to prevent overlap

        # Save the figure as an image file in the specified directory
        fig.savefig(f'../images/color_quiver/day_{day:02d}.png', format='png')
        plt.close(fig)  # Close the figure to free memory

# Function to compile saved images into a GIF animation
def create_gif():
    images = []
    for day in day_list:
        # Read each saved image and append it to the list for GIF creation
        images.append(imageio.imread(f'../images/color_quiver/day_{day:02d}.png'))
    
    # Save all frames into a single GIF file with a specified frame rate
    imageio.mimsave(f'../gifs/color_quiver_{num_frames}.gif', images, fps=2)

# Generate the images and create the GIF
generate_images()
create_gif()
print("Completed")

Running...
Completed
